## Diffecientwo

> By deut-erium
> 
> Welcome to the Diffecientwo Caching Database API for tracking and storing content across social media. We have repurposed our security product, as saving the admin key was probably not the best idea.
> 
> We have decided to change our policies and to achieve better marketing, we are offering free API KEY to customers sharing #SEKAICTF #DEUTERIUM #DIFFECIENTWO #CRYPTO on LonelyFans (our premium business partner).
>
> `nc chals.sekai.team 3000`
>
> Attachment: diffecientwo.py

A server implements a bloom filter with murmurhash3 using fixed known seeds of 0-63. We can add 22 new items of at most length 32 to the bloom filter and need to manipulate the bloom filter such that the length 42 magic string `#SEKAICTF #DEUTERIUM #DIFFECIENTWO #CRYPTO` is in the filter.

We want to generate some messages which when passed throught the bloom filter collide with at least 3[^1] of the hashes derived from the magic string. We can try to cause this collision by solving for inputs using z3. One key observation for speedup is to only constrain collisions only immediately after the `xor length` operation in murmurhash3 as the bit shift operations after that are not easily invertible.

[^1]: 'round(64/22)`

In [1]:
import random
import z3

def chunk(x, bs=4):
    return [
        x[i:i+bs]
        for i in range(0, len(x), bs)
    ]

# ====================== pure impl ======================

def mask(x, m=0xffffffff):
    return x & m

def rotl32(x, n):
    return mask((x << n) | (x >> (32-n)))

def mmh3_block(h, data):
    k = data
    k = mask(k * 0xcc9e2d51)
    k = rotl32(k, 15)
    k = mask(k * 0x1b873593)

    h = h ^ k
    h = rotl32(h, 13)
    h = mask(h * 5 + 0xe6546b64)

    return h

def mmh3_tail(h, tail):
    k = tail
    k = mask(k * 0xcc9e2d51)
    k = rotl32(k, 15)
    k = mask(k * 0x1b873593)
    
    h = h ^ k

    return h

def mmh3_addlen(h, total_len):
    return h ^ total_len

def mmh3_finalize(h):
    h = h ^ (h >> 16)
    h = mask(h * 0x85ebca6b)
    h = h ^ (h >> 13)
    h = mask(h * 0xc2b2ae35)
    h = h ^ (h >> 16)

    return h

def mmh3(data, tail=None, tail_len=0, seed=0x0):
    if isinstance(data, bytes):
        data = chunk(data)
        if len(data[-1]) != 4:
            data, tail = data[:-1], data[-1]
            tail_len = len(tail)

        data = [
            int.from_bytes(data_i, "little")
            for data_i in data   
        ]
        if tail is not None:
            tail = int.from_bytes(tail, "little")

    trace = {
        "blocks": [], 
        "tail": None,
        "addlen": None,
        "final": None
    }

    total_len = len(data) * 4 + tail_len
    
    cur_hash = seed
    for data_i in data:
        cur_hash = mmh3_block(cur_hash, data_i)
        trace["blocks"].append(cur_hash)

    if tail is not None:
        cur_hash = mmh3_tail(cur_hash, tail)
        trace["tail"] = cur_hash
    
    cur_hash = mmh3_addlen(cur_hash, total_len)
    trace["addlen"] = cur_hash

    cur_hash = mmh3_finalize(cur_hash)
    trace["final"] = cur_hash

    return trace

# ====================== symbolics ======================

def rotl32_sym(x, n):
    new_left, new_right = z3.Extract(31-n, 0, x), z3.Extract(31, 31-n+1, x)
    return z3.Concat(new_left, new_right)

def mmh3sym_block(h, data):
    k = data
    k = k * 0xcc9e2d51
    k = rotl32_sym(k, 15)
    k = k * 0x1b873593

    h = h ^ k
    h = rotl32_sym(h, 13)
    h = h * 5 + 0xe6546b64

    return h

def mmh3sym_tail(h, tail):
    k = tail
    k = k * 0xcc9e2d51
    k = rotl32_sym(k, 15)
    k = k * 0x1b873593
    
    h = h ^ k

    return h

def mmh3sym_addlen(h, total_len):
    return h ^ total_len

def mmh3sym_finalize(h):
    h = h ^ z3.LShR(h, 16)
    h = h * 0x85ebca6b
    h = h ^ z3.LShR(h, 13)
    h = h * 0xc2b2ae35
    h = h ^ z3.LShR(h, 16)

    return h

def mmh3sym(data, tail=None, tail_len=0, seed=0x0, solver=None):
    nonce = random.randbytes(8).hex()
    def save_bv(suffix, eqn):
        new_bv = z3.BitVec(f"mmh3_{nonce}_{suffix}", 32)
        solver.add(new_bv == eqn)
        return new_bv
    
    trace = {
        "blocks": [], 
        "tail": None,
        "addlen": None,
        "final": None
    }

    total_len = len(data) * 4 + tail_len
    cur_hash = seed
    for i, data_i in enumerate(data):
        cur_hash = mmh3sym_block(cur_hash, data_i)

        cur_hash = save_bv(f"b{i}", cur_hash)
        trace["blocks"].append(cur_hash)

    if tail is not None:
        cur_hash = mmh3sym_tail(cur_hash, tail)

        cur_hash = save_bv("t", cur_hash)
        trace["tail"] = cur_hash

    cur_hash = mmh3sym_addlen(cur_hash, total_len)
    cur_hash = save_bv("l", cur_hash)
    trace["addlen"] = cur_hash
    
    cur_hash = mmh3sym_finalize(cur_hash)

    cur_hash = save_bv("f", cur_hash)
    trace["final"] = cur_hash

    return trace

In [2]:
magic = b"#SEKAICTF #DEUTERIUM #DIFFECIENTWO #CRYPTO"

addlen_goals = []
for seed in range(0, 64):
    addlen_goals.append((seed, mmh3(magic, seed=seed)["addlen"]))
print(addlen_goals)

[(0, 2538403748), (1, 2698667000), (2, 2611445724), (3, 2587362448), (4, 3649470961), (5, 2971484104), (6, 2747869548), (7, 3492042989), (8, 2606332996), (9, 2747428504), (10, 2743148796), (11, 2986842032), (12, 2911794580), (13, 2642185064), (14, 2625066508), (15, 2640236928), (16, 979172321), (17, 807813925), (18, 567584793), (19, 544136797), (20, 3563947569), (21, 899816693), (22, 537779881), (23, 3587084205), (24, 3503235329), (25, 898412613), (26, 2732155324), (27, 2608626288), (28, 420642385), (29, 2641628200), (30, 3069223756), (31, 3058078144), (32, 2382468388), (33, 2218383480), (34, 2227654236), (35, 2271820048), (36, 2410567028), (37, 3852968520), (38, 3938292204), (39, 3957175392), (40, 876735937), (41, 3616723461), (42, 878061177), (43, 421494845), (44, 925310993), (45, 3517574357), (46, 2236544396), (47, 3935633408), (48, 2752631396), (49, 3996735416), (50, 3984468892), (51, 4020206544), (52, 2562434996), (53, 3935391752), (54, 3029993260), (55, 2601059872), (56, 24077725

In [3]:
from multiprocessing import Pool
from tqdm.auto import tqdm

def solve_slice(addlen_slice):
    s = z3.Solver()
    n_chunks = 6
    d_chunks = [z3.BitVec(f"d{i}", 32) for i in range(n_chunks)]

    for j, (_, addlen) in enumerate(addlen_slice):
        # print(f"Solving seed={j} => addlen={addlen}")
        s.add(mmh3sym(d_chunks, seed=j, solver=s)["addlen"] == addlen)
    if str(s.check()) != "sat":
        return [i[0] for i in addlen_slice], "err"

    m = s.model()
    d_hex = "".join([m[i].as_long().to_bytes(4, "little").hex() for i in d_chunks])
    return [i[0] for i in addlen_slice], d_hex

collisions = []
with Pool(16) as p:
    slices = chunk(addlen_goals, 3)
    for res in tqdm(p.imap_unordered(solve_slice, slices), total=len(slices)):
        collisions.append(res)
        print(res)

  0%|          | 0/22 [00:00<?, ?it/s]

([15, 16, 17], '42cef17b4ab3914caa86fcd4132bcffe03034c0fd3cd151c')
([21, 22, 23], 'c7915c2ec5f0db9d427b05c9c90fae0feb40d40be24b3dd0')
([45, 46, 47], '0643d2cfb6b85605b114b34b54af84ed155b684ce487ec2c')
([30, 31, 32], '6b86b777986f37df94e9c10aa3dc4b55cd7ccd032c86a207')
([24, 25, 26], '49fc091621b7060584547b341b6975444d979c1181a25ae7')
([0, 1, 2], 'ebdbe651d60a7cc3a42cbb26448e587aab494e6b0cd98a26')
([63], 'ce55c8f2061a0a3bcd32f20b2711dcd1d5b3ce62f4bb4897')
([36, 37, 38], '99d06c7fb227fd3dd553532f31c268578e4b96761c510d9e')
([9, 10, 11], '98eff98f0bac953b92e337153fbcd4ab5e301226a7ce663f')
([3, 4, 5], '13a180bb5c26510922eb0af67841ba40dd3559bdf2ff20ea')
([6, 7, 8], '64e17ffd878c46d812617ea61e9edd1a1363f5e73bb047e9')
([39, 40, 41], 'fc1b50b8727c1ba1cb920804801d2a46d9f1eef72ad7d433')
([33, 34, 35], '6f6a24b7ee163935c1f48aecf1396e67af96506985245acf')
([54, 55, 56], 'fe32ca63e9d2a3a737ef462aa3e7955fe059c1aac1cbb0cf')
([51, 52, 53], '9677651246c8bace2dcba8a25206aea221f676033e0a0893')
([18, 19, 20]

In [10]:
from pwn import *

payload = []
for _, i in collisions:
    payload.append("2")
    payload.append(i)
payload.append("3")
payload = "\n".join(payload)

r = remote("chals.sekai.team", 3000)
r.sendline(payload.encode())
r.recvuntil(b"}").rpartition(b"\n")[2]

b"b'SEKAI{1c8d8986a4c530225cb7711c4064dd63f19350ebdc46a250b9a8af16893203d8}"